# Prepare the DUDE database for analysis

Perform preprocessing on the DUDE database.  Assumes that the DUDE database has been downloaded and placed in the path "data/dude/unprocessed". Here, we expect directories for each target containing 'actives_final.ism' and 'decoys_final.ism'.
It may be beneficial to split the below out of this Jupyter notebook and deploy to HPC or a non interactive environment. To pre-cache descriptors, run cache_mols_from_csv() on the DudePreprocessor object below.

The step may also be skipped by downloading the OpenFEPOPS-DUDE data directory created for analysis of the DUDE diversity set from FigShare at https://doi.org/10.6084/m9.figshare.23951445.v1 .  See the OpenFEPOPS README.md for further information.

In [ ]:
from typing import Union, Optional
from pathlib import Path
from tqdm import tqdm
from fepops import OpenFEPOPS
import pandas as pd
import multiprocessing as mp
from rdkit import Chem


class DudePreprocessor:
    def __init__(
        self,
        *,
        dude_directory: Union[Path, str] = "data/dude/",
    ) -> None:
        print(dude_directory)
        print(type(dude_directory))
        self.dude_path = Path(dude_directory)
        self.dude_unprocessed_path = self.dude_path / Path("unprocessed")
        if not self.dude_path.exists():
            raise FileNotFoundError(f"Dude dataset not found in path: {self.dude_path}")
        self.dude_processed_path = self.dude_path / Path("processed")
        self.dude_processed_path.parent.mkdir(parents=True, exist_ok=True)
        self.fepops_ob = OpenFEPOPS()

    def __call__(
        self,
    ):
        self.process()

    def process(
        self,
        targets: Optional[Union[str, list[str]]] = None,
        skip_existing: bool = True,
    ):
        if targets is None:
            dude_targets = [
                t.parent.name
                for t in self.dude_path.glob("unprocessed/*/actives_final.ism")
            ]
        else:
            if isinstance(targets, str):
                targets = [targets]
        print(f"Processing the following DUDE targets: {dude_targets}")
        for target in tqdm(dude_targets, desc=f"Preparing targets"):
            self.create_dude_target_csv_data(target, skip_existing=skip_existing)

    @staticmethod
    def _parallel_init_worker_desc_gen_shared_fepops_ob():
        global shared_fepops_ob
        shared_fepops_ob = OpenFEPOPS()

    @staticmethod
    def _parallel_get_rdkit_cansmi(s):
        global shared_fepops_ob
        mol = shared_fepops_ob._mol_from_smiles(s)
        if mol is None:
            return ""
        return Chem.MolToSmiles(mol)

    def create_dude_target_csv_data(
        self,
        dude_target: Path,
        actives_file: Path = Path("actives_final.ism"),
        decoys_file: Path = Path("decoys_final.ism"),
        seperator: str = " ",
        skip_existing: bool = True,
    ):
        target_output_file = self.dude_processed_path / f"dude_target_{dude_target}.csv"
        if skip_existing and target_output_file.exists():
            print(
                f"Found existing {target_output_file}, skipping due to skip_existing = True, rerun as False to regenerate"
            )
        actives = pd.read_csv(
            self.dude_unprocessed_path / Path(dude_target) / actives_file,
            sep=seperator,
            header=None,
            names=["SMILES", "DUDEID", "CHEMBLID"],
        )
        actives["Active"] = 1
        decoys = pd.read_csv(
            self.dude_unprocessed_path / Path(dude_target) / decoys_file,
            sep=seperator,
            header=None,
            names=["SMILES", "DUDEID"],
        )
        decoys["Active"] = 0
        df = pd.concat([actives, decoys]).reset_index().drop(columns="index")
        df["rdkit_canonical_smiles"] = tqdm(
            mp.Pool(
                initializer=self._parallel_init_worker_desc_gen_shared_fepops_ob
            ).imap(self._parallel_get_rdkit_cansmi, df.SMILES, chunksize=100),
            desc=f"Generating {dude_target} benchmark file",
            total=len(df),
        )
        df.to_csv(target_output_file, index=False)

    def cache_mols_from_csv(
        self,
        csv_path: Union[Path, str],
        rdkit_canonical_smiles_column_header: str = "rdkit_canonical_smiles",
    ):
        """Cache mols from a CSV into a db for faster recall later

        Parameters
        ----------
        csv_path : Union[Path, str]
            Path of CSV file. If None, then all CSV files in the DUDE datasets
            processed path are used.
        rdkit_canonical_smiles_column_header : str, optional
            _description_, by default "rdkit_canonical_smiles"
        """

        from fepops.fepops_persistent import get_persistent_fepops_storage_object

        for csv_path in (
            [Path(csv_path)]
            if csv_path is not None
            else self.dude_processed_path.glob("dude_target_*.csv")
        ):
            df = pd.read_csv(csv_path)
            if df[rdkit_canonical_smiles_column_header].isnull().values.any():
                print(
                    f"Whilst working on caching {csv_path}, the following mol rows did not contain RDKit canonical SMILES:"
                )
                print(df[df[rdkit_canonical_smiles_column_header].isnull()])
            smiles = [
                s
                for s in df[rdkit_canonical_smiles_column_header].tolist()
                if not pd.isnull(s)
            ]
            with get_persistent_fepops_storage_object(csv_path.with_suffix(".db")) as f:
                f.save_descriptors(smiles)

dude_preprocessor=DudePreprocessor()
dude_preprocessor()

# Analyse the DUDE diversity set and obtain mean and standard deviations for each 'standard' set of FEPOPS descriptors

Below, we read in the DUDE diversity set and gather the mean and standard deviation of the produced FEPOPS descriptors and use these values as defaults for scaling (before scoring) all FEPOPS descriptors. Before running this, there should be 8 targets:

* akt1
* ampc
* cp3a4
* cxcr4
* gcr
* hivpr
* hivrt
* kif11

represented by their associated .csv and .db files present in the data/dude/processed/diversity_set/ directory

In [27]:
from pathlib import Path
from fepops import OpenFEPOPS
from tqdm import tqdm
from fepops.fepops_persistent import get_persistent_fepops_storage_object
import numpy as np
dude_diversity_set_path=Path("data/dude/processed/diversity_set/")
diversity_target_files=list(dude_diversity_set_path.glob("dude_target_*.csv"))
print(diversity_target_files)
descriptors=[]
for diversity_target in diversity_target_files:
    f=get_persistent_fepops_storage_object(diversity_target.with_suffix(".db"))
    print(f"Working on {diversity_target.stem}")
    for (orig_smi, dude_id, chemblid, active_flag, can_smi) in tqdm([l.strip().split(",") for l in open(diversity_target).readlines()[1:] if len(l)>3]):
        status, retrieved_descriptors=f.get_fepops(can_smi)
        if status.value ==1:
            for d in retrieved_descriptors:
                descriptors.append(d)
            
descriptors=np.array(descriptors)
display("Mean:", descriptors.mean(axis=0))
display("Std:", descriptors.std(axis=0))

[PosixPath('data/dude/processed/diversity_set/dude_target_kif11.csv'), PosixPath('data/dude/processed/diversity_set/dude_target_hivrt.csv'), PosixPath('data/dude/processed/diversity_set/dude_target_cxcr4.csv'), PosixPath('data/dude/processed/diversity_set/dude_target_ampc.csv'), PosixPath('data/dude/processed/diversity_set/dude_target_gcr.csv'), PosixPath('data/dude/processed/diversity_set/dude_target_cp3a4.csv'), PosixPath('data/dude/processed/diversity_set/dude_target_akt1.csv'), PosixPath('data/dude/processed/diversity_set/dude_target_hivpr.csv')]
Working on dude_target_kif11


  0%|          | 0/6966 [00:00<?, ?it/s]

100%|██████████| 6966/6966 [00:05<00:00, 1255.33it/s]


Working on dude_target_hivrt


100%|██████████| 19229/19229 [00:13<00:00, 1378.07it/s]


...

100%|██████████| 36286/36286 [00:30<00:00, 1176.21it/s]


'Mean:'

array([-0.28932319,  0.5166312 ,  0.37458883,  0.99913668, -0.04193182,
        1.03616917,  0.27327129,  0.99839024,  0.09701198,  1.12969387,
        0.23718642,  0.99865705,  0.35968991,  0.6649304 ,  0.4123743 ,
        0.99893657,  5.70852885,  6.3707943 ,  6.47354071,  6.26385429,
        6.19229367,  6.22946713])

'Std:'

array([0.35067291, 1.00802116, 0.48380817, 0.02926675, 0.15400475,
       0.86220776, 0.44542581, 0.03999429, 0.16085455, 0.92042695,
       0.42515847, 0.03655217, 0.35778578, 1.36108994, 0.49210665,
       0.03252466, 1.96446927, 2.30792259, 2.5024708 , 2.4155645 ,
       2.29434487, 2.31437527])

# Benchmark FEPOPS using the DUDE diversity set
Collect AUROC scores and compare against Morgan 2 and RDKit fingeprints
<hr>

Perform imports, define similarity methods (except OpenFEPOPS which will be defined upon target assessment in order to load cached descriptors)

In [4]:
import time
import numpy as np
from tqdm import tqdm
from rdkit.Chem import rdMolDescriptors

from fepops import OpenFEPOPS
from dataclasses import dataclass
from typing import Callable, Union, Optional
from pathlib import Path
import pandas as pd
from rdkit import Chem

from rdkit.Chem import rdMolDescriptors
from rdkit.Chem import AllChem
from rdkit.Chem import DataStructs
from sklearn.metrics import roc_auc_score
from fepops.fepops_persistent import get_persistent_fepops_storage_object
from typing import Union, Optional
from fepops import OpenFEPOPS
from fepops.fepops_persistent import get_persistent_fepops_storage_object
from fepops.fepops import GetFepopStatusCode
import json
import sys

open_fepops_object=OpenFEPOPS()

@dataclass
class SimilarityMethod:
    name: str
    supports_multiple_candidates: bool
    descriptor_calc_func: Optional[Callable] = None
    descriptor_score_func: Optional[Callable] = None


# OpenFEPOPS will be added to this as a persistent object, reading from a DB of cached
# molecules for each target in the diversity set
similarity_methods = {
    'Morgan 2': SimilarityMethod(
        "Morgan 2",
        False,
        lambda x: AllChem.GetMorganFingerprint(x, 2),
        lambda x, y: DataStructs.TanimotoSimilarity(x,y),
    ),
    'MACCS': SimilarityMethod(
        "MACCS",
        False,
        lambda x: rdMolDescriptors.GetMACCSKeysFingerprint(x),
        lambda x, y: DataStructs.TanimotoSimilarity(x,y),
    ),
    'RDKit': SimilarityMethod(
        "RDKit",
        False,
        lambda x: Chem.RDKFingerprint(x,maxPath=4),
        lambda x, y: DataStructs.TanimotoSimilarity(x,y),
    ),
}

diversity_set_csv_files = list(Path("data/dude/processed/diversity_set/").glob("dude_target*.csv"))

# Write all CSVs to SMILES files
print(f"Got {len(diversity_set_csv_files)} diversity_set_csv_files : {[f.stem for f in diversity_set_csv_files]}")
for csv_file_path in diversity_set_csv_files:
    pd.read_csv(csv_file_path,sep=",",
        index_col=[0],
        header=0,
        ).loc[:,['rdkit_canonical_smiles', 'DUDEID']].reset_index().to_csv(csv_file_path.with_suffix(".smi"), sep=" ", index=None, header=False)
    

Got 8 diversity_set_csv_files : ['dude_target_kif11', 'dude_target_hivrt', 'dude_target_cxcr4', 'dude_target_ampc', 'dude_target_gcr', 'dude_target_cp3a4', 'dude_target_akt1', 'dude_target_hivpr']


Perform AUROC score calculation

In [ ]:
auroc_scores_info_df=pd.DataFrame()

for csv_file_path in diversity_set_csv_files:
    
    # We replace the OpenFepops similarity object at each new diversity CSV file
    # so that new databases may be loaded for speed of descriptor retrieval.
    ofepops_persistent=get_persistent_fepops_storage_object(csv_file_path.with_suffix(".db"))
    similarity_methods['OpenFEPOPS']=SimilarityMethod(
        "OpenFEPOPS",
        True,
        lambda x: ofepops_persistent.get_fepops(x, is_canonical=True),
        ofepops_persistent.calc_similarity,
    )
    
    df=pd.read_csv(csv_file_path,sep=",",
                index_col=[0],
                header=0,
            ).reset_index()
    print(df.head())
    descriptors={k:[] for k in similarity_methods.keys()}
    smiles_list = df['rdkit_canonical_smiles'].tolist()
    labels_list = df['Active'].astype(int).tolist()
    problematic_compound_indexes=[]
    for sm_name, sm in similarity_methods.items():
        # Cache all descriptors for each molecular similarity technique, as some mols may be bad and have to be removed
        for smiles_i, smiles in tqdm(enumerate(smiles_list), desc=f"Caching {sm_name} descriptors for {csv_file_path.stem}"):
            mol_from_smiles=open_fepops_object._mol_from_smiles(smiles)
            if mol_from_smiles is None:
                problematic_compound_indexes.append(smiles_i)
                descriptors[sm_name].append(np.nan)
            else:
                res = sm.descriptor_calc_func(mol_from_smiles)
                if isinstance(res, tuple):
                    if res[0]==GetFepopStatusCode.FAILED_RETRIEVED_NONE or res[0]==GetFepopStatusCode.FAILED_TO_GENERATE or res[0]==GetFepopStatusCode.FAILED_TO_RETRIEVE or res[1] is None:
                        print(f"Problem with {smiles}, {res}")
                        problematic_compound_indexes.append(smiles_i)
                        descriptors[sm_name].append(np.nan)
                    else:                    
                        descriptors[sm_name].append(res[-1])
                else:
                    if res is None:
                        problematic_compound_indexes.append(smiles_i)
                    descriptors[sm_name].append(res)
    # Remove failed molecules from pool of descriptors and labels
    for k,v in descriptors.items():
        descriptors[k]=[v[ii] for ii in range(len(v)) if ii not in problematic_compound_indexes]
    labels_list=[labels_list[ii] for ii in range(len(labels_list)) if ii not in problematic_compound_indexes]
    auroc_scores={smn:[] for smn in similarity_methods.keys()}
    
    info=pd.Series(dtype=object)
    for sm_name, sm in similarity_methods.items():
        # Remove entries which did not return a mol
        info['target']=csv_file_path.stem.replace("dude_target_","")
        info['similarity_method']=sm_name
        info['smiles_count']=len(smiles_list)
        info['actives_count']=np.sum(labels_list)
        info['failed_smiles']=len(problematic_compound_indexes)
        info['failed_active_smiles']=len([ft for ft in problematic_compound_indexes if labels_list[ft]==1])
        for active_i in tqdm(
                np.argwhere(np.array(labels_list) == 1).flatten(),
                desc=f"Assessing active recall (AUROC) for {sm.name}",
            ):
            if sm.supports_multiple_candidates:
                scores = np.array(
                    sm.descriptor_score_func(
                        descriptors[sm_name][active_i], descriptors[sm_name]
                    ),
                    dtype=float,
                ).flatten()
            else:
                scores = np.array(
                    [
                        sm.descriptor_score_func(
                            descriptors[sm_name][active_i], descriptors[sm_name][smiles_i]
                        )
                        for smiles_i in range(len(descriptors[sm_name]))
                    ],
                    dtype=float,
                ).flatten()
            
            auroc_scores[sm_name].append(roc_auc_score(
                np.array(labels_list)[np.argwhere(~np.isnan(scores))],
                scores[np.argwhere(~np.isnan(scores))],
                )
            )
        info['average_auroc_score']=np.mean(auroc_scores[sm_name])
        info['median_auroc_score']=np.median(auroc_scores[sm_name])
        info['q1_auroc_score']=np.percentile(auroc_scores[sm_name],0.25)
        info['q3_auroc_score']=np.percentile(auroc_scores[sm_name],0.75)
        auroc_scores_info_df=pd.concat([auroc_scores_info_df, info.to_frame().T], ignore_index=True, axis=0)
    print("Writing to ", csv_file_path.parent)

    json.dump(auroc_scores,open(csv_file_path.parent/Path(f"res_scores_{csv_file_path.stem}.json"),"w"))
auroc_scores_info_df.to_csv(csv_file_path.parent/Path(f"res_df_{csv_file_path.stem}.csv"))
print(auroc_scores_info_df)
    

Output summary AUROC results table as shown in the paper

In [9]:
display(pd.concat([pd.read_csv(f) for f in Path("data/dude/processed/diversity_set/").glob("res_df*.csv")]).pivot(values='average_auroc_score',index='target', columns=['similarity_method'],)[['Morgan 2', 'RDKit', 'MACCS', 'OpenFEPOPS']])

similarity_method,Morgan 2,RDKit,MACCS,OpenFEPOPS
target,,,,
akt1,0.835717,0.833538,0.741306,0.828947
ampc,0.783629,0.659837,0.673310,0.639115
cp3a4,0.602774,0.613335,0.581545,0.649807
cxcr4,0.697226,0.592485,0.854251,0.898971
gcr,0.670138,0.708350,0.665839,0.616173
hivpr,0.779689,0.759308,0.681237,0.677882
hivrt,0.651011,0.660054,0.669939,0.583981
kif11,0.763058,0.672460,0.667887,0.713152


Output an extended table with information on failed molecules etc

In [28]:
display(pd.concat([pd.concat([pd.read_csv(f) for f in Path("data/dude/processed/diversity_set/").glob("res_df*.csv")]).pivot(values='average_auroc_score',index='target', columns=['similarity_method'],),pd.concat([pd.read_csv(f) for f in Path("data/dude/processed/diversity_set/").glob("res_df*.csv")]).query("similarity_method=='OpenFEPOPS'")[['target','smiles_count','actives_count','failed_smiles','failed_active_smiles']].set_index("target")], axis=1))

,MACCS,Morgan 2,OpenFEPOPS,RDKit,smiles_count,actives_count,failed_smiles,failed_active_smiles
target,,,,,,,,
akt1,0.741306,0.835717,0.828947,0.833538,16743,293,0,0
ampc,0.673310,0.783629,0.639115,0.659837,2898,48,1,0
cp3a4,0.581545,0.602774,0.649807,0.613335,11970,170,4,0
cxcr4,0.854251,0.697226,0.898971,0.592485,3446,40,0,0
gcr,0.665839,0.670138,0.616173,0.708350,15258,258,4,0
hivpr,0.681237,0.779689,0.677882,0.759308,36286,535,1,1
hivrt,0.669939,0.651011,0.583981,0.660054,19229,338,9,0
kif11,0.667887,0.763058,0.713152,0.672460,6966,116,0,0
